[矢吹太朗『Webのしくみ』（サイエンス社, 2020）](https://github.com/taroyabuki/webbook)

下のアイコンをクリックすれば，この文書に掲載されているコードを，Google Colab上で実行できます．（Googleのアカウントが必要です．）

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taroyabuki/webbook/blob/master/chapters/08_cipher.ipynb)

# 第8章 暗号


## 8.1 事後否認・改ざん・なりすまし・盗聴

## 8.2 暗号の基本

![図8.1 シーザー暗号（「+13」文字ずらす）を使って，アリスがボブにメッセージを送っている様子](https://raw.githubusercontent.com/taroyabuki/webbook/master/chapters/figures/08-1.svg)

In [ ]:
#準備
!apt-get -y install hxtools | tail -n 1

In [ ]:
!echo "hello" | rot13

In [ ]:
!echo "hello" | rot13 | rot13

## 8.3 公開鍵暗号

### 8.3.1 盗聴への対策

アリスはボブの公開鍵で暗号化し，ボブはボブの秘密鍵で復号します．

![図8.2 公開鍵暗号を使った暗号化と復号](https://raw.githubusercontent.com/taroyabuki/webbook/master/chapters/figures/08-2.svg)

ファイル|役割
--|--
bob-public.pem|ボブの公開鍵
bob-private.pem|ボブの秘密鍵
message4bob|ボブへのメッセージ
message4bob-secret|ボブへの暗号文


In [ ]:
#(1) ボブが公開鍵と秘密鍵を作る．
!openssl genrsa -out bob-private.pem 1024
!openssl rsa -in bob-private.pem -pubout -out bob-public.pem

In [ ]:
#公開鍵を見てみる．
!cat bob-public.pem

In [ ]:
#秘密鍵を見てみる．
!cat bob-private.pem

In [ ]:
#(2) ボブが公開鍵を公開する．

#(3) アリスがボブの公開鍵を手に入れる．

#(4) アリスがボブの公開書きを使ってメッセージを暗号化し，ボブに送る．

#(4.1) メッセージをファイルに書く．
!echo "こんにちは" > message4bob

#(4.2) ボブの公開鍵を使って暗号文を作る．
!openssl pkeyutl -encrypt -pubin -inkey bob-public.pem -in message4bob -out message4bob-secret
!cat message4bob-secret

In [ ]:
#(5) ボブが暗号文を受け取り，秘密鍵を使って復号する．
!openssl pkeyutl -decrypt -inkey bob-private.pem -in message4bob-secret

### 8.3.2 デジタル署名

アリスはアリスの秘密鍵で「暗号化」し，ボブはアリスの公開鍵で「復号」します．

![図8.3 RSA暗号の特徴の一つ：秘密鍵で「暗号化」し，公開鍵で復号することもできる．](https://raw.githubusercontent.com/taroyabuki/webbook/master/chapters/figures/08-3.svg)

ファイル|役割
--|--
alice-public.pem|アリスの公開鍵
alice-private.pem|アリスの秘密鍵
alice-message|アリスのメッセージ
alice-message-secret|アリスの「暗号文」

#### 「暗号文」

In [ ]:
#(1) アリスが公開鍵と秘密鍵を作る．
!openssl genrsa -out alice-private.pem 1024
!openssl rsa -in alice-private.pem -pubout -out alice-public.pem

In [ ]:
#(2) アリスがメッセージをファイルに書き，アリスが秘密鍵を使って「暗号文」を作る．
!echo -n "HELLO" > alice-message
!openssl pkeyutl -sign -inkey alice-private.pem -in alice-message -out alice-message-secret

In [ ]:
#(3) アリスが「暗号文」と公開鍵を公開する．

#(4) ボブがアリスの「暗号文」と公開鍵を取得する．

#(5) ボブがアリスの公開鍵を使って「暗号文」を復号する．
!openssl pkeyutl -verifyrecover -pubin -in alice-message-secret -inkey alice-public.pem

#### ハッシュ値の「暗号化」（説明用）

ファイル|役割
--|--
alice-public.pem|アリスの公開鍵
alice-private.pem|アリスの秘密鍵
alice-message|アリスのメッセージ
alice-hash|アリスのメッセージのハッシュ値
alice-hash-secret|アリスのメッセージのハッシュ値の「暗号文」

In [ ]:
#アリスのメッセージのハッシュ値
!openssl md5 alice-message > alice-hash
!cat alice-hash

In [ ]:
#改ざんされたメッセージのハッシュ値
!echo -n "HELL0" | openssl md5

In [ ]:
#(1) アリスが，秘密鍵を使ってハッシュ値の「暗号文」を作る．
!openssl pkeyutl -sign -inkey alice-private.pem -in alice-hash -out alice-hash-secret

In [ ]:
#(2) アリスが，メッセージ，「暗号文」，公開鍵を公開する．

#(3) ボブが，アリスのメッセージ，「暗号文」，公開鍵を取得する．

#(4) ボブが，アリスのメッセージのハッシュ値を求める．
!openssl md5 alice-message

In [ ]:
#(5) ボブが，アリスの公開鍵を使って「暗号文」を復号する．
!openssl pkeyutl -verifyrecover -pubin -in alice-hash-secret -inkey alice-public.pem

#### デジタル署名（実用）

ファイル|役割
--|--
alice-public.pem|アリスの公開鍵
alice-private.pem|アリスの秘密鍵
alice-message|アリスのメッセージ
alice-signature|アリスのメッセージのデジタル署名



In [ ]:
#(1) アリスが，秘密鍵を使ってメッセージのデジタル署名を作る．
!openssl dgst -md5 -sign alice-private.pem -out alice-signature alice-message

In [ ]:
#(2) アリスが，メッセージ，デジタル署名，公開鍵を公開する．

#(3) ボブが，アリスのメッセージ，デジタル署名，公開鍵を取得する．

#(4) ボブが，アリスの公開鍵を使って，アリスのメッセージとデジタル署名を検証する．
!openssl dgst -md5 -verify alice-public.pem -signature alice-signature alice-message

### 8.3.3 デジタル証明書

## 8.4 ウェブの安全性

### 8.4.1 TLS